In [1]:
!pip install pandas-datareader
!pip install --upgrade pandas
!pip install --upgrade pandas-datareader



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 109 kB 6.6 MB/s 
  Attempting uninstall: pandas-datareader
    Found existing installation: pandas-datareader 0.9.0
    Uninstalling pandas-datareader-0.9.0:
      Successfully uninstalled pandas-datareader-0.9.0


In [2]:
!pip install tensorflow-gpu==2.10.0
!pip install tensorflow==2.10.0


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 578.0 MB 15 kB/s 
     |████████████████████████████████| 438 kB 67.4 MB/s 
     |████████████████████████████████| 5.9 MB 46.2 MB/s 
     |████████████████████████████████| 1.7 MB 58.3 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully u

In [3]:
!pip install pyodbc

import pyodbc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 472 kB 6.5 MB/s 


In [ ]:
!pip install --upgrade numpy

In [3]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as data_reader



from tqdm import tqdm_notebook, tqdm
from collections import deque

tf.__version__


'2.1.0'

In [4]:
class AI_Trader():
    def __init__(self, state_size, action_space = 3, model_name = "AITrader"):
        self.state_size = state_size
        self.action_space = action_space
        self.memory = deque(maxlen = 2000)
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_final = 0.01
        self.epsilon_decay = 0.995
        self.model = self.model_builder()

    def model_builder(self):
        model = tf.keras.models.Sequential()
        model.add(tf.keras.layers.Dense(units = 32, activation = "relu", input_dim = self.state_size))
        model.add(tf.keras.layers.Dense(units = 64, activation = "relu"))
        model.add(tf.keras.layers.Dense(units = 128, activation = "relu"))
        model.add(tf.keras.layers.Dense(units = self.action_space, activation = "linear"))
        model.compile(loss = "mse", optimizer = tf.keras.optimizers.Adam(lr = 0.001))
        return model

    def trade(self, state):
        if random.random() <= self.epsilon:
            return random.randrange(self.action_space)
    
        actions = self.model.predict(state)
        return np.argmax(actions[0])

    def batch_train(self, batch_size):
        batch = []
        ranger = range(len(self.memory) - batch_size + 1, len(self.memory))
        
        for i in ranger:
            batch.append(self.memory[i])
            
        #print(next_state)
            
        for state, action, reward, next_state, done in batch:
            if not done:
                amax = np.amax(self.model.predict(next_state)[0])
                reward = reward + self.gamma * amax

            target = self.model.predict(state)
            target[0][action] = reward

            self.model.fit(state, target, epochs = 1, verbose = 0)

        if self.epsilon > self.epsilon_final:
            self.epsilon *= self.epsilon_decay

def sigmoid(x):   
    x = x/100
    #print("#############################")
    #print(x) 
    return 1/(1+math.exp(-x))

def stocks_price_format(n):
    if n < 0:
        return "- $ {0:2f}".format(abs(n))
    else:
        return "$ {0:2f}".format(abs(n))

def dataset_loader(stock_name):
    dataset = data_reader.DataReader(stock_name, data_source = "yahoo")
    start_date = str(dataset.index[0]).split()[0]
    end_date = str(dataset.index[-1]).split()[0]
    close = dataset['Close']
    return close

def state_creator( data, timestep, window_size):    
    starting_id = timestep- window_size + 1

    if starting_id >= 0:
        windowed_data = data[starting_id:timestep +1]
    else:
        windowed_data = -starting_id * [data[0]] + list(data[0:timestep + 1])
  
    state = []
    for i in range(window_size - 1):
        #print("##################")
        #print(windowed_data[i + 1]) 
        #print(windowed_data[i])
        state.append(sigmoid(windowed_data[i + 1] - windowed_data[i]))
        
    #print(state)
    #print(windowed_data)

    return np.array([state]), np.array(windowed_data)

In [5]:
stock_name = "AAPL"
data = dataset_loader(stock_name)
s, w = state_creator(data, 0, 5)

window_size = 10
episodes = 1000
batch_size = 32
data_samples = len(data) - 1


In [ ]:
trader = AI_Trader(window_size)

for episode in range(1, episodes + 1):
  print("Episode: {}/{}".format(episode,episodes))
  state = state_creator(data, 0, window_size + 1)
  total_profit = 0
  trader.inventory = []
  for t in tqdm(range(data_samples)):
    action = trader.trade(state)
    next_state = state_creator(data, t + 1, window_size +1)
    reward = 0

    if action == 1:
      trader.inventory.append(data[t])
      print("AI Trader bought: ", stocks_price_format(data[t]))

    elif action == 2 and len(trader.inventory) > 0:
      buy_price = trader.inventory.pop(0)

      reward = max(data[t] - buy_price, 0)
      total_profit += data[t] - buy_price
      print("AI Trader sold: ", stocks_price_format(data[t]), " Profit: " + stocks_price_format(data[t] - buy_price))

    print(str(t) + "/" + str(data_samples))
    if t == data_samples - 1:
      done = True
    else:
      done = False
    
    trader.memory.append((state, action, reward, next_state, done))
    state = next_state

    if done:
      print("##################")
      print("Total profit: {}".format(total_profit))
      print("##################")

    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)

    if episode % 10 == 0:
      trader.model.save("ai_trader_{}.h5".format(episode))




Episode: 1/1000


  0%|          | 0/1257 [00:00<?, ?it/s]

0/1257
1/1257
2/1257
AI Trader bought:  $ 43.970001
3/1257
4/1257
AI Trader sold:  $ 43.492500  Profit: - $ 0.477501
5/1257
6/1257
7/1257
8/1257
9/1257
AI Trader bought:  $ 42.494999
10/1257
11/1257
AI Trader bought:  $ 43.740002
12/1257
13/1257
14/1257
AI Trader sold:  $ 43.267502  Profit: $ 0.772503
15/1257
AI Trader sold:  $ 42.369999  Profit: - $ 1.370003
16/1257
17/1257
AI Trader bought:  $ 42.762501
18/1257
AI Trader sold:  $ 42.450001  Profit: - $ 0.312500
19/1257
20/1257
AI Trader bought:  $ 42.252499
21/1257
AI Trader bought:  $ 42.330002
22/1257
23/1257
AI Trader sold:  $ 43.167500  Profit: $ 0.915001
24/1257
25/1257
26/1257
27/1257
AI Trader sold:  $ 43.492500  Profit: $ 1.162498
28/1257
29/1257
30/1257
31/1257
32/1257


  3%|▎         | 33/1257 [00:02<01:14, 16.34it/s]

33/1257
AI Trader bought:  $ 42.642502
34/1257


...

931/1257


 74%|███████▍  | 932/1257 [34:34<13:41,  2.53s/it]

932/1257


 74%|███████▍  | 933/1257 [34:36<13:02,  2.42s/it]

933/1257
